In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import mplfinance as mpf
import talib as tb

tickerSymbol = 'AAPL'
tickerData = yf.Ticker(tickerSymbol)
df = tickerData.history(period="1d", start='2007-3-8', end='2023-3-8')

df.drop(['Stock Splits', 'Dividends'], axis=1, inplace=True)
df

df["RSI"] = tb.RSI(df['Close'].values, timeperiod=14)
df['SMA20'] = tb.SMA(df['Close'], timeperiod=20)
df['SMA50'] = tb.SMA(df['Close'], timeperiod = 50)

df.columns

resistance_levels = []
support_levels = []

def isResistance(df,i):
    return df['High'][i] > df['High'][i-1] and df['High'][i] > df['High'][i+1] and df['High'][i+1] > df['High'][i+2] and df['High'][i-1] > df['High'][i-2]

def isSupport(df,i):
    return df['Low'][i] < df['Low'][i-1]  and df['Low'][i] < df['Low'][i+1] and df['Low'][i+1] < df['Low'][i+2] and df['Low'][i-1] < df['Low'][i-2]
  

for i in range(2,df.shape[0]-2):
    if isSupport(df,i):
        support_levels.append((i,df['Low'][i]))
    elif isResistance(df,i):
        resistance_levels.append((i,df['High'][i]))   

print(len(resistance_levels))
print(len(support_levels))

avg_candlesize = np.mean(df['High'] - df['Low'])  

# def isFarFromLevel(l):
#    return np.sum([abs(l-x) < s for x in levels or abs(l-x) < s for x in levelr]) == 0

# function to check if a level is far from existing levels
def isFarFromLevel(l):
    # check if l is far from support levels
    far_from_support = np.sum([abs(l-x) < avg_candlesize for x in updated_support_levels]) == 0
    # check if l is far from resistance levels
    far_from_resistance = np.sum([abs(l-x) < avg_candlesize for x in updated_resistance_levels]) == 0
    return far_from_support or far_from_resistance

updated_support_levels = []
updated_resistance_levels = []

for res in resistance_levels:
    if isFarFromLevel(res[1]):
        updated_resistance_levels.append(res)
        
for sup in support_levels:
    if isFarFromLevel(sup[1]):
        updated_support_levels.append(sup)        

print(len(updated_support_levels))
print(len(updated_resistance_levels))

df['Resistance'] = 0
df['Support'] = 0

for index, row in df.iterrows():
    high_value = row['High']
    if any(high_value == url[1] for url in updated_resistance_levels):
        df.at[index, 'Resistance'] = 1
    
    low_value = row['Low']
    if any(low_value == usl[1] for usl in updated_support_levels):
        df.at[index,'Support'] = 1
        

for index, row in df.iterrows():
    if row['Support']==1:
        print(row["Low"])

for index, row in df.iterrows():
    if row['Resistance']==1:
        print(row["High"])

df['Signal'] = np.where((df['SMA20'] > df['SMA50'])  & (df['Support']==1), 1,np.where((df['SMA20'] < df['SMA50']) & (df['Resistance']==1), -1, 0))

df[df['Signal'] == 1]

df[df['Signal'] == -1]

df = df.reset_index(drop=True)

# Initialize variables
positions = []
profit = []
investment = 0

# Loop through the DataFrame
for i, row in df.iterrows():
       # Check if it's a buy signal
    if row['Signal'] == 1:
        # Buy at the open price of the next day
        buy_price = df.loc[i+1, 'Open']
        positions.append({'buy_price': buy_price})
        
    # Check if it's a sell signal
    elif row['Signal'] == -1:
        # Check if there are any open positions
        if len(positions) > 0:
            # Sell all open positions at the close price of the current day
            for pos in positions:
                investment += pos['buy_price']
                profit.append(row['Close'] - pos['buy_price'])
            positions = []
            
# Print the total profit
# print('Total profit: {:.2f}'.format(profit))

df.index

print(investment,sum(profit))

profit